In [ ]:
#Import Dependencies
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt
import os

In [ ]:
#Import Dependencies
import numpy as np
import pandas as pd
import datetime as dt

In [ ]:
#Import Flask
from flask import Flask

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
import os
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, text, inspect
from sqlalchemy.ext.declarative import declarative_base

In [ ]:
# create engine to hawaii.sqlite
#engine = create_engine("sqlite:///../Resources/hawaii.sqlite")

# Define the directory and database filename separately
directory = "/Users/marvinalarry/Documents/Education/Tulsa Community College/Data Visualization/Homework/Challenge #10 - Grade_WIP/sqlalchemy-challengeMarvinaLarry/Surfsup/Resources"
db_filename = "hawaii.sqlite"

# Combine them to create the full database file path
db_file_path = os.path.join(directory, db_filename)

# Create an SQLAlchemy engine object
engine = create_engine(f"sqlite:///{db_file_path}")

In [ ]:
# Check if the file exists
# if os.path.exists(db_file_path):
#     print(f"The file {db_filename} exists at the path {db_file_path}.")
# else:
#     print(f"The file {db_filename} does not exist at the path {db_file_path}.")

In [ ]:
#print(os.getcwd())

In [ ]:
# Create the inspector and connect it to the engine
inspector = inspect(engine)

In [ ]:
# Collect the names of tables within the database
table = inspector.get_table_names()

In [ ]:
# reflect an existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(autoload_with=engine)

In [ ]:
# View all of the classes that automap found
# class_names = Base.classes.keys()
# print(class_names)
Base.classes.keys()

In [ ]:
# Save references to each table
measurements = Base.classes.measurement
stations = Base.classes.station

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Precipitation Analysis

In [ ]:
# Find the most recent date in the data set.
#find_recent_date = session.query(func.max(measurements.date)).first()
find_recent_date = session.query(measurements.date).order_by(measurements.date.desc()).first()
find_recent_date

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 


# Calculate the date one year from the last date in data set.
last_date_PY = dt.date(2017,8,23)- dt.timedelta(days = 365)
#print(last_date_PY)

# Perform a query to retrieve the date and precipitation scores
retrieve_QRY = session.query(measurements.date, measurements.prcp).filter(measurements.date >= last_date_PY).all()
#print(retrieve_QRY)

# Save the query results as a Pandas DataFrame. Explicitly set the column names
pdate_df = pd.DataFrame(retrieve_QRY, columns = ["date", "precipitation"])

# Sort the dataframe by date
sorted_df = pdate_df.sort_values("date")
#print(sorted_df)

In [ ]:
# Use Pandas Plotting with Matplotlib to plot the data
sorted_df.plot(x="date", y="precipitation",rot=90)
plt.ylabel("inches")

# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number of stations in the dataset
station_count_QRY = session.query(func.count(stations.station)).all()
#station_count_QRY

In [ ]:
# select station_name, count(station_name) from measurements
# group by station_name
# order by 2

In [ ]:
# Design a query to find the most active stations (i.e. which stations have the most rows?)
# List the stations and their counts in descending order.
active_station_QRY = session.query(measurements.station,func.count(measurements.station)).group_by(measurements.station).order_by(func.count(measurements.station).desc()).all()
active_station_QRY

In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.
HiLowAvg = session.query(func.min(measurements.tobs),func.max(measurements.tobs),func.avg(measurements.tobs)).filter(measurements.station == "USC00519281").all()
HiLowAvg

In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
last12tobs_QRY = session.query(measurements.tobs).filter(measurements.station == "USC00519281").filter(measurements.date >= last_date_PY).all()
last12tobs_QRY

tempsdata_df = pd.DataFrame(last12tobs_QRY, columns = ["tobs"])
tempsdata_df.plot(kind = "hist", bins = 12)

# Close Session

In [ ]:
# Close Session
session.close()